#Collaborative Filtering using deep learning
Vineet Kothari
50291159
vineetsu@buffalo.edu


##Ultimate cell

In [0]:
import torch 
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import os
from PIL import Image
import glob
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import matplotlib.pyplot as plt
import random
from bokeh.io import curdoc, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from functools import partial
from threading import Thread
from tornado import gen
import time
import pickle
from torchsummary import summary
from tqdm import tqdm
from functools import reduce
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
#parameters
batch_size=100
train_split=0.8
num_epochs=5






In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import os
import datetime
import random

In [45]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
#paths
root_dir="/content/drive/My Drive/Projects/Collaborative filtering/"
fcodes_dataset_path="/content/drive/My Drive/Projects/Collaborative filtering/Datasets/fcodesbucket edited - fcodesbucket.csv"
version="dae-3"
ckpt_dir="/content/drive/My Drive/Projects/Collaborative filtering/Implementations/dae-3/checkpoints/"
model_dir="/content/drive/My Drive/Projects/Collaborative filtering/Implementations/dae-3/checkpoints/"

In [47]:
text=open(fcodes_dataset_path).read()
print("Text is {} characters long".format(len(text)))

Text is 381175 characters long


In [49]:
histories=[h for h in text.split('\n')]
print("Data has {} patient histories".format(len(histories)))
n_patients=len(histories)

Data has 6237 patient histories


In [50]:
n_patients=len(histories)

real_data=[]
for history in histories:
  real_data.append([symptom for symptom in history.split(',')])
for history in real_data:
  while "" in history:
    history.remove("")
  history.append("End")

real_data[0]

['H269', 'D259', 'F067', 'End']

Sorting the symptoms, except the last 'end'

In [0]:
for i,_ in enumerate(real_data):
  real_data[i]=sorted(real_data[i][:-1])+[real_data[i][-1]]

Splitting each symptom in to 3 levels
e.g. A423 as A, 42, 3

Splitting one history into multiple sub histories
H, 20, 4, G,12,6 into
[H],[H,20],[H,20,4],[H,20,4,G]

Min length has to be 4.
i.e. One whole symptom has to be there (Not split, whole with all 3 levels) 


In [52]:
data=[]
for history in real_data:
  hist=[]
  for s in history:
    if s!="End":
      try:
        hist.extend([s[0],s[1:3],s[3]])
      except:
        hist.extend([s[0],s[1:3],'0'])
    else:
      hist.append(s)
  data.append(hist)
data[0]



['D', '25', '9', 'F', '06', '7', 'H', '26', '9', 'End']

In [53]:
data[0]

['D', '25', '9', 'F', '06', '7', 'H', '26', '9', 'End']

Combining all histories to one

In [0]:
text=[]
for history in data:
  text.extend(history)


Each characted mapped as a no.

In [55]:
vocab=sorted(set(text))
print("There are {} unique characters".format(len(vocab)))
char2int={c:i for i,c in enumerate(vocab)}
int2char=np.array(vocab)
print("Vector:\n")
for char,_ in zip(char2int,range(8)):
  print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 132 unique characters
Vector:

 '0' :   0,
 '00':   1,
 '01':   2,
 '02':   3,
 '03':   4,
 '04':   5,
 '05':   6,
 '06':   7,


In [56]:
char2int['End']

115

##Data analysis

In [0]:
def flatten2D(grid):
  a=[]
  for x in grid:
    a.extend(x)
  return a

Mapping each of the level codes in histories to nos.

In [58]:
int_data=[]
for history in data:
  int_data.append(np.array([char2int[level_code] for level_code in history],dtype=np.int32))
int_data=np.array(int_data)
print ('{}\n mapped to integers:\n {}'.format(repr(data[0]), int_data[0]))

['D', '25', '9', 'F', '06', '7', 'H', '26', '9', 'End']
 mapped to integers:
 [113  28  99 116   7  77 118  29  99 115]


Creating training and validation sets

In [0]:
n_histories=len(data)

BLinding it with no 420

In [60]:
#Find the max length of history
max_history_len=0
for history in int_data:
  max_history_len=max(len(history),max_history_len)
print("Longest history of a patient: ",max_history_len)

Longest history of a patient:  94


In [0]:
# #Find the max length of history
# max_history_len=0
# for history in encode_data:
#   max_history_len=max(len(history),max_history_len)
# print("Longest history of a patient: ",max_history_len)

In [0]:
# def random_blinding(history):
#   n=len(history)
#   history=list(map(lambda x:x/132,history))
#   y_history=history.copy()
#   req_len=95-n
#   res=[]
#   for it in range(2*n):
#     temp=history.copy()
#     blind_indices = random.sample(range(0, n), round(n*0.3)) 
#     for i in blind_indices:
#       temp[i]=1   
#     #Padding shoudl be done here. since, 
    
#     temp=np.append(temp,[(char2int['End'])/132]*req_len)
#     y_history=np.append(history.copy(),[(char2int['End'])/132]*req_len)
    
#     res.append([temp,y_history])
#   return res

This is complete blinding. But needs to be optimized as the RAM limits the operation. Cool no problem as of now!

In [0]:
# print("Blinding the integer codes of diseases with no. 420")
# print("BLINDING PERCENT 30%.\n ")

# aug_int_data=[]
# for history in int_data:
#   aug_int_data.extend(random_blinding(history))
# n_aug_histories=len(aug_int_data)
# #aug_int_data=np.array(aug_int_data)

# print("Length of complete data points:",len(aug_int_data)) # blinded data
# print("Example of a blinded data pointL: ",aug_int_data[0])

In [0]:
def encode_single_vector(vector):
  res_vector=[]
  for value in vector:
    encoded_value=[0]*132
    if value!=132:
      encoded_value[value]=1
    res_vector.append(encoded_value)
  return res_vector

def one_hot_encode(int_data):
  res=[]
  padded_input=[0]*132
  padded_input[char2int['End']]=1
  for history in int_data:
    n=len(history)
    #y_history=history.copy()
    req_len=95-n
    res.append(encode_single_vector(history)+[padded_input]*req_len)
  return res

def random_blinding(history):
  n=len(history)
  #y_history=history.copy()
  req_len=95-n
  res=[]
  for it in range(2*n):
    temp=history.copy()
    blind_indices = random.sample(range(0, n), round(n*0.3)) 
    for i in blind_indices:
      temp[i]=[0]*132   
    #Padding shoudl be done here. since, 
    padded_input=[0]*132
    padded_input[char2int['End']]=1
    #temp=np.append(temp,[padded_input]*req_len)
    #y_history=np.append(history.copy(),[padded_input]*req_len)
    
    res.append([np.append(temp,[padded_input]*req_len),np.append(history.copy(),[padded_input]*req_len)])
  return res

In [65]:
encode_data=one_hot_encode(int_data)
print("Dimensions of encoded data are: ({},{},{})".format(len(encode_data),len(encode_data[0]),len(encode_data[0][0])))

Dimensions of encoded data are: (6237,95,132)


In [0]:
# print("Blinding the integer codes of diseases with zeros")
# print("BLINDING PERCENT 30%.\n ")

# aug_encode_data=[]
# for history in encode_data:
#   aug_encode_data.extend(random_blinding(history))
# n_aug_histories=len(aug_encode_data)
# #aug_int_data=np.array(aug_int_data)

# print("Length of complete data points:",len(aug_encode_data)) # blinded data
# print("Example of a blinded data pointL: ",aug_encode_data[0])

TODO: https://discuss.pytorch.org/t/where-is-the-noise-layer-in-pytorch/2887/6

##From Tensorflow doc - Creating dataset
https://www.tensorflow.org/tutorials/load_data/text

Pytorch GRU with attention maps
https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py

##Pytorch code starts here.

In [0]:
# class listDataset(torch.utils.data.Dataset):
#   def __init__(self,aug_int_data):
#     self.aug_int_data=aug_int_data
     
#     self.data_transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize([0.485],[0.229]) # need to normalize . How to do this will find out!.
#     ]) # oh my god! I also need to pad the input! Cool. # shoudl I do it before? would be easy.
    
#   def __getitem__(self,index):
    
#     blind_input=(self.aug_int_data[index][0]) # Rememeber each data point is a numpy array. Indexing might be different
#     real_input=(self.aug_int_data[index][1]) #need to put the data variable. 
#     return blind_input,real_input
  
#   def __len__(self):
#     return len(self.aug_int_data)
  


In [0]:
def single_random_blind(history):
  n=len(history)
  blind_indices = random.sample(range(0, n), round(n*0.3)) 
  for i in blind_indices:
    history[i]=[0]*132
  return history   

In [0]:
class listDataset(torch.utils.data.Dataset):
  def __init__(self,encode_data):
    self.encode_data=encode_data

  def __getitem__(self,index):
    temp1,temp2=self.encode_data[index].copy(),self.encode_data[index].copy()
    blind_input=flatten2D(single_random_blind(temp1)) 
    real_input=flatten2D(temp2) 
    return torch.Tensor(blind_input),torch.Tensor(real_input)
  
  def __len__(self):
    return len(self.encode_data)

In [0]:
dataset=listDataset(encode_data)

In [0]:
# dataset=listDataset(aug_int_data)

In [71]:
#example of an iteration of dataloader
len(iter(torch.utils.data.DataLoader(dataset)).next()[1])

1

Splitting into training and validation set.

In [72]:
print("Splitting the dataloader into training and validation..\n")
train_size=int(train_split*len(dataset))
val_size=len(dataset)-train_size
batch_size=64
train_dataset, val_dataset=torch.utils.data.random_split(dataset,[train_size,val_size])
train_dataloader=torch.utils.data.DataLoader(dataset=train_dataset,
                                             batch_size=batch_size,
                                             shuffle=True)
num_batches=len(train_dataloader)
val_dataloader=torch.utils.data.DataLoader(dataset=val_dataset,
                                             batch_size=batch_size,
                                             shuffle=True)
print("Length of train dataset :{}, of val dataset:{}".format(
    len(train_dataset),len(val_dataset)
))

Splitting the dataloader into training and validation..

Length of train dataset :4989, of val dataset:1248


In [73]:
iter(train_dataloader).next()[0].shape
#this means there are 2 lists, blind and real of 12540 tensors. and each tensor is of batch_size. BOOM


torch.Size([64, 12540])

In [74]:
len(train_dataset[1][1])

12540

Model

In [0]:
class Autoencoder(nn.Module):
  def __init__(self,input_size,hidden_size,latent_rep_size):
    super(Autoencoder,self).__init__()
    self.fc1= nn.Linear(input_size,hidden_size)
    self.relu1=nn.ReLU()
    self.fc2=nn.Linear(hidden_size,latent_rep_size)
    self.relu2=nn.ReLU()
    self.fc3=nn.Linear(latent_rep_size,hidden_size)
    self.relu3=nn.ReLU()
    self.fc4= nn.Linear(hidden_size,input_size)
    self.t=nn.Sigmoid()

  def weight_init(self, mean=0.0, std=0.02):
        for m in self._modules:
            normal_init(self._modules[m]) 

  def forward(self,x):
    out=self.fc1(x)
    out=self.relu1(out)
    out=self.fc2(out)
    out=self.relu2(out)
    out=self.fc3(out)
    out=self.relu3(out)
    out=self.fc4(out)
    return self.t(out)


def normal_init(m, mean=0.0, std=0.02):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [0]:
#parameters
lr=0.002
beta1=0.5
beta2=0.999
L1_lambda=1

In [77]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device="cpu"
print("Training on device: ", device)
input_size=12540
hidden_size=300
latent_rep_size=128
model=Autoencoder(input_size,hidden_size, latent_rep_size).to(device)
print(model)

model.load_state_dict(torch.load(model_dir+'model.ckpt'))
model.eval()
#Loss and optimizer
criterion=nn.MSELoss().to(device)
#optimizer=torch.optim.Adam(model.parameters(),lr=lr,betas=(beta1,beta2))

Training on device:  cuda:0
Autoencoder(
  (fc1): Linear(in_features=12540, out_features=300, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=300, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=300, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=300, out_features=12540, bias=True)
  (t): Sigmoid()
)


In [0]:
def single_bittensor_acc(tensor1, tensor2):
  tensor1,tensor2=tensor1.detach().round().int(),tensor2.detach().round().int()
  return sum(tensor1^tensor2)

def bittensor_acc(tensor1,tensor2):
  sum_acc,n=0,len(tensor1)
  for t1,t2 in zip(tensor1,tensor2):
    sum_acc+=single_bittensor_acc(t1,t2)
  return sum_acc/n

def  decode_onehot(vector):
  if 1 in vector:
    return list(vector).index(1)
  else:
    return -1

def decode_disease(vector):
  history=list(vector.detach().round().int()).copy()
  res=[]
  unknown_count=0
  for i in range(0,12540,132):
    temp=history[i:i+132].copy()
    index=decode_onehot(list(temp))
    if index!=-1 and index!=115:
      res.append(int2char[index])
    elif index==115:
      res.append(int2char[index])
      break
    else:
      unknown_count+=1
      res.append('*')
  return unknown_count,res



In [0]:
# num_epochs=100
# total_step=len(train_dataloader)
# #model.train()
# for epoch in range(num_epochs):
#   for i,((blind_vector,real_vector),(bv_val,rv_val)) in enumerate(zip(train_dataloader,val_dataloader)):
#     blind_vector=blind_vector.to(device).float()
#     real_vector=real_vector.to(device).float() 
#     bv_val=bv_val.to(device).float()
#     rv_val=rv_val.to(device).float() 

#     outputs=model(blind_vector)
#     outputs_val=model(bv_val)
#     loss=criterion(outputs,real_vector)
#     loss_val=criterion(outputs_val,rv_val)
    
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     if(i+1)%10==0:
#       print('Epoch [{}/{}], Step[{}/{}],Training:: Loss:{:.4f},Accuracy:{}         Validation Loss:{:.4f}, Accuracy:{}'
#            .format(epoch+1,num_epochs,i+1,total_step,loss.item(),bittensor_acc(outputs,real_vector),loss_val.item(),bittensor_acc(outputs_val,rv_val)))



In [88]:
with torch.no_grad():
  blind_vector, real_vector=iter(val_dataloader).next()
  blind_vector=blind_vector.to(device).float()
  real_vector=real_vector.to(device).float()
  index=8
  outputs=model(blind_vector)
  loss=criterion(outputs,real_vector)
  print("Predicted:",decode_disease(model(blind_vector)[index]))
  print("Blinded Input:",decode_disease(blind_vector[index]))
  print("Groundtruth:",decode_disease(real_vector[index]))
  print("Loss:{:.4f}, Avg. no of bits misclassified: {}, Recall :{:.4f}, Precision :{:.4f}, F1 :{:.4f}"
        .format(loss.item(),bittensor_acc(outputs,real_vector),1.0,1.0,1.0 )
  )

(5, ['F', '32', '*', '*', '22', '*', 'K', '63', '5', 'K', '80', '2', 'N', '80', '*', 'R', '*', '0', 'End'])
(7, ['*', '32', '*', '*', '22', '7', 'K', '63', '*', 'K', '80', '2', 'N', '84', '*', 'R', '51', '*', '*', 'End'])
(0, ['F', '32', '9', 'K', '22', '7', 'K', '63', '5', 'K', '80', '2', 'N', '84', '1', 'R', '51', '0', 'End'])
Loss:0.0006, Avg. no of bits misclassified: 8, Recall :1.0000, Precision :1.0000, F1 :1.0000


Hierarchial model

RACT Collaborative filtering

Journal search VMC,Sciom data mining 



In [86]:
precision_recall_fscore_support(flatten2D(list(outputs.cpu().round())),flatten2D(list(real_vector.cpu().round())))

(array([0.99989202, 0.92828947]),
 array([0.99945283, 0.98499127]),
 array([0.99967238, 0.95580017]),
 array([796830,   5730]))

In [87]:
confusion_matrix(flatten2D(list(outputs.cpu().round())),flatten2D(list(real_vector.cpu().round())))

array([[796394,    436],
       [    86,   5644]])